# Random gates

## Setup configurations

In [ ]:
import numpy as np
from quantify_core.measurement import MeasurementControl
from quantify_scheduler import Schedule
from quantify_scheduler.device_under_test.transmon_element import BasicTransmonElement
from quantify_scheduler.gettables import ScheduleGettable
from quantify_scheduler.device_under_test.quantum_device import QuantumDevice
from quantify_scheduler.instrument_coordinator import InstrumentCoordinator
from quantify_scheduler.instrument_coordinator.components.qblox import ClusterComponent, PulsarQRMComponent
from quantify_scheduler.operations.gate_library import Measure, X, X90, Y, Reset
from quantify_scheduler.resources import ClockResource
from quantify_scheduler.operations.pulse_library import IdlePulse, SquarePulse
from quantify_scheduler.operations.acquisition_library import SSBIntegrationComplex, Trace
from quantify_scheduler.enums import BinMode
from qcodes.instrument.parameter import ManualParameter
from qblox_instruments import Cluster, ClusterType, Pulsar
from quantify_core.data import handling as dh
from quantify_scheduler import compilation
from quantify_scheduler.backends import SerialCompiler
import math
import random

In [ ]:
dh.set_datadir(dh.default_datadir(verbose=False));

## Setup cluster

In [ ]:
cluster = Cluster("cluster0",
                  "<ip>",
                  debug=1,
                  dummy_cfg={15: ClusterType.CLUSTER_QRM},
          )

ic_cluster0 = ClusterComponent(cluster)
my_instr_coord = InstrumentCoordinator("my_instr_coord")
my_instr_coord.add_component(ic_cluster0)

In [ ]:
ic_cluster0.name

In [ ]:
my_device = QuantumDevice("my_device")

In [ ]:
transmon0 = BasicTransmonElement(f"q0")
transmon0.clock_freqs.readout(10e9)
transmon0.clock_freqs.f01(10e9)
transmon0.clock_freqs.f12(10e9)
transmon0.rxy.amp180(0.1)
transmon0.rxy.motzoi(0.1)
#transmon0.measure.acq_duration(2e-08)
transmon0.measure.pulse_duration(1e-07)
transmon0.measure.acq_delay(1.2e-07)
transmon0.measure.integration_time(2e-08)

my_device.add_element(transmon0)
transmon1 = BasicTransmonElement(f"q1")
transmon1.clock_freqs.readout(10e9)
my_device.add_element(transmon1)

In [ ]:
my_device.instr_instrument_coordinator("my_instr_coord")

In [ ]:
hw_config = {
        "backend": "quantify_scheduler.backends.qblox_backend.hardware_compile",
        "cluster0": {
                "ref": "internal",
                "instrument_type": "Cluster",
                "cluster0_module15": {
                        "instrument_type": "QRM",
                        "complex_output_0": {
                                "portclock_configs": [
                                        {"port": "q0:res", "clock": "q0.ro", "interm_freq": 0},
                                        {"port": "q1:res", "clock": "q1.ro", "interm_freq": 0},
                                        {"port": "q0:mw", "clock": "q0.01", "interm_freq": 0},
                                        {"port": "q1:mw", "clock": "q1.01", "interm_freq": 0},
                                    ]
                            }
                    }
            },
    }

In [ ]:
my_device.hardware_config(hw_config)
my_device.cfg_sched_repetitions(1)

## Setup schedule, dummy data

In [ ]:
signal_delay = 148e-9
num_acquisitions=1000

In [ ]:
from qblox_instruments import DummyBinnedAcquisitionData, DummyScopeAcquisitionData

In [ ]:
def set_dummy_data(repetitions=1):
    dummy_slot_idx = 15
    cluster.delete_dummy_binned_acquisition_data(slot_idx=dummy_slot_idx)
    dummy_data_0 = [
            DummyBinnedAcquisitionData(data=(4 * i, 0), thres=0, avg_cnt=0) for i in range(num_acquisitions)
    ] * repetitions
    cluster.set_dummy_binned_acquisition_data(slot_idx=dummy_slot_idx, sequencer=0, acq_index_name="0", data=dummy_data_0)
    dummy_data_1 = [
            DummyBinnedAcquisitionData(data=(0, 8 * i), thres=0, avg_cnt=0) for i in range(num_acquisitions)
    ] * repetitions
    cluster.set_dummy_binned_acquisition_data(slot_idx=dummy_slot_idx, sequencer=1, acq_index_name="1", data=dummy_data_1)

In [ ]:
def create_schedule(
    repetitions: int,
) -> Schedule:
    OPERATIONS = (X, X90, Y)
    
    sched = Schedule('Complex Sequencing')
    sched.add(Reset('q0'))
    for idx in range(5000):
        gate = random.choice(OPERATIONS)
        operation = sched.add(gate('q0'))
    
    sched.add(Measure('q0'))
    sched.repetitions = 1
    
    return sched

In [ ]:
simple_binned_acquisition_kwargs = {
}

## Run schedule and profiling

In [ ]:
def run_experiment():
    my_gettable = ScheduleGettable(
        my_device,
        create_schedule,
        simple_binned_acquisition_kwargs,
        batched=True,
        real_imag=False,
        data_labels=[f"q0 abs", f"q0 phase", f"q1 abs", f"q1 phase"],
        num_channels=2,
    )
    acq = my_gettable.get()
    return acq